In [1]:
import subprocess
import os
def istooclose(input_line,dict_of_name_info):
    for info in dict_of_name_info:
        if info.split()[0] == input_line[9]:
            added_info = dict_of_name_info[info].split(',')
    
    min_dist = 10000

    if added_info[7] == "query":
        query_coords = added_info[3:6]
    else:
        query_coords = added_info[0:3]
    query_coords[1] = int(query_coords[1])
    query_coords[2] = int(query_coords[2])
    if query_coords[0] != input_line[13]:
        return False #can't be too close if on seperate chroms
    return (not (int(query_coords[2])+min_dist < int(input_line[15])+1 or int(input_line[16]) < int(query_coords[1])-min_dist+1))
def check_aln_length(input_line):
    #get length of query segment
    query_len = int(input_line[12])-(int(input_line[11])+1)
    target_len = int(input_line[16]) - (int(input_line[15])+1)
    return abs(query_len-target_len)
def stitch_segments(input_line):
    join_length = 20000000000000
    query_coords,target_coords,join_query, join_target = [],[],[],[]
    lengths = [int(x) for x in input_line[18][0:-1].split(",")]
    query_starts = [int(x) for x in input_line[19][0:-1].split(",")]
    target_starts = [int(x) for x in input_line[20][0:-1].split(",")]
    #print(lengths, query_starts, target_starts)
    
    if input_line[8] == "+":
        for i in range(len(lengths)):#these coords are currently in 0based notation but are corrected below
            query_coords.append([query_starts[i]+1,query_starts[i]+lengths[i]])
            target_coords.append([target_starts[i]+1,target_starts[i]+lengths[i]])
    else: 
        length_of_query = int(input_line[10])
        for i in range(len(lengths)):
            query_start = length_of_query +1 - query_starts[i] -lengths[i]
            query_end = length_of_query - query_starts[i] 
            #print(query_start)
            query_coords.append([query_start,query_end])
            
            target_coords.append([target_starts[i]+1,target_starts[i]+lengths[i]])
        query_coords.reverse()
    for i in range(len(query_coords)-1):
        if query_coords[i+1][0] - query_coords[i][1] +1 > join_length:
            join_query.append(False)
        else:
            join_query.append(True)
        if target_coords[i+1][0] - target_coords[i][1] +1 > join_length:
            join_target.append(False)
        else:
            join_target.append(True)
    if False in join_target or False in join_query:
        return "", "", ""
    
    return join_query, join_target, query_coords
  
allow_chroms = []
for i in range(0,39):
    allow_chroms.append(f"chr{i}")
allow_chroms.append("chrX")

In [2]:
#set constants
minmatch = 25
#sdust_path = "/home/blacksmi/links/kidd-lab-umms/jmkidd-projects/ballard-assemblies/run-minigraphpan/analysis/progs/sdust/sdust"

def process_masked_fasta(file_name):
    seq_dict = {}
    current_name = ''
    current_seq = ''
    infile = open(file_name,'rt')
    for line in infile.readlines():
        #print(line)
        if line.startswith(">"):
            line = line[1::]
            if current_name == "":
                current_name = line.rstrip()
            else:
                seq_dict[current_name] = current_seq
                current_seq = ""
                current_name = line.rstrip()
        else:
            current_seq = current_seq + line.rstrip()
    seq_dict[current_name] = current_seq
    infile.close()
    
    
    #print(seq_dict)
    
    dict_of_N = {}
    for key in seq_dict.keys():
        sequence = seq_dict[key]
        #print(sequence)
        current = []
        current.append([])
        #print(len(current[-1]))
        for i in range(len(sequence)):
            if sequence[i] == "N" or sequence[i] == "n":
                if len(current[-1]) == 0:
                    current[-1].append(i+1)
                elif len(current[-1]) == 1:
                    continue
                elif len(current[-1]) == 2:
                    current.append([i+1])
                else:
                    print("Shouldn't be here")
                    break
            else:
                if len(current[-1]) == 0:
                    continue
                elif len(current[-1]) == 1:
                    current[-1].append(i)
                elif len(current[-1]) == 2:
                    continue
                else:
                    print("Also shouldn't be here")
                    break
        if len(current[-1]) == 1:
            current[-1].append(i+1)
        dict_of_N[key] = current
    return dict_of_N, seq_dict
def identify_candidate_regions_with_unmasked_bases(input_dict):
    valid_locus = 0
    for key in input_dict.keys():
        total_unmasked = 0
        #print(input_dict[key])
        for nuc in input_dict[key]:
            if nuc != "N" and nuc != "n":
                total_unmasked+=1
        if total_unmasked >= 25:
            valid_locus+=1
    return valid_locus

def process_unmasked_file(input_file):
    #this is necessary because the unmasked fasta has all of the necessary information which includes the original locus, and the coordinates in mCanlor.
    input_file = input_file + ".filtered"
    with open(input_file,'rt') as infile:
        dict_of_names = {}
        for line in infile:
            if not line.startswith(">"):
                continue
            line = line[1::].split()
            dict_of_names[line[0]] = line[1]
    return dict_of_names


def identify_high_copy_repeats(infile):
    cmd = f"cat {infile} | cut -f 10 | uniq -c"
    proc = subprocess.run(cmd, shell=True, capture_output=True,text=True)
    output = proc.stdout.split('\n')[5::]
    #print(output)
    pass_loci = []
    for locus in output[0:-1]:
        locus = locus.split()
        #print(locus)
        if int(locus[0]) > 50:
            pass_loci.append(locus[1])
    #print(pass_loci)
    return pass_loci
def format_for_output(loci_dict,output_list,dict_of_locus_name_info,canine,state):
    if len(output_list) == 0:
        header = "Comparison_Genome\tState\tLocus_chrom\tRef_start\tRef_end\tPost_element_coords\tTD_coords\tOrientation\n"
        output_list.append(header)
    for line in loci_dict.keys():
        name_info = dict_of_locus_name_info[f"{canine}_{state}"][line].split(",")
        #print(name_info)
        for transduction in loci_dict[line]:
            output_line = [canine,state,name_info[3],name_info[4],name_info[5],line,transduction[0],transduction[3]]
            output_line = "\t".join(output_line) + "\n"
            output_list.append(output_line)


    return output_list
dict_of_dicts = {}
dict_of_locus_name_info = {}
canines = ["Mischka","Nala","Tasha","Zoey","Sandy","Clu"]
states = ['query','ref']
Elibible_loci = []
Final_TDs = []
list_of_output_lines = []
for canine in canines:
    for state in states:
        masked_fasta = f"/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/Revised_All_Canines_List_2025_08_06/Transduction_Detection/{canine}_mCanlor_{state}_fasta_extract.fa_filtered.masked"
        dict_of_locus_name_info[f"{canine}_{state}"] = process_unmasked_file(masked_fasta[0:-16])
        N_loci, seq_dict = process_masked_fasta(masked_fasta)
        valid_locus = identify_candidate_regions_with_unmasked_bases(seq_dict)
        skipped_loci = 0
        infile = f"/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/Revised_All_Canines_List_2025_08_06/Transduction_Detection/{canine}_mCanlor_{state}_fasta_extract.filtered.pslx"
        high_copy = identify_high_copy_repeats(infile)
        #break
        dict_of_loci = {}
        with open(infile,'rt') as infile:
            for line in infile.readlines()[5:]:
                split_line = line.split()
                if split_line[9] in high_copy:
                    continue
                if int(split_line[0]) < minmatch:
                    continue
                if split_line[13] not in allow_chroms:
                    continue
                if istooclose(split_line,dict_of_locus_name_info[f"{canine}_{state}"]):
                    continue
                if check_aln_length(split_line) > 100:
                    continue
                #print(split_line)
                join_query, join_target, query_coords = stitch_segments(split_line)
                #print(join_query,join_target, query_coords)
                if query_coords == "":
                    skipped_loci +=1
                    #print('skipped')
                    continue

                dict_of_finals = {}
                for query_index in range(len(query_coords)):
                    overlap_coords = []
                    if len(N_loci[split_line[9]][-1]) == 0:
                        final_segments = query_coords
                        continue
                    for j in range(len(N_loci[split_line[9]])):#This loop is going to be for iterating over the relevant sections of the mask
                        overlap_bool = (not (query_coords[query_index][1] < N_loci[split_line[9]][j][0] or N_loci[split_line[9]][j][1] < query_coords[query_index][0]))
                        #print(overlap_bool)
                        if overlap_bool == True:
                            overlap_coords.append([max(query_coords[query_index][0],N_loci[split_line[9]][j][0]),min(query_coords[query_index][1],N_loci[split_line[9]][j][1])])
                    #iterate over the overlap to remove those regions from the query coords and then check what is left.
                    final_segments = []
                    if len(overlap_coords) == 0:
                        final_segments = query_coords
                        #break
                    for i in range(len(overlap_coords)):
                        if len(overlap_coords) == 0:
                            final_segments = query_coords
                            break
                        if i == 0:
                            if overlap_coords[0][0] > query_coords[query_index][0]:
                                final_segments.append([query_coords[query_index][0],overlap_coords[0][0]-1])
                            elif overlap_coords[0][0] == query_coords[query_index][0]:
                                if len(overlap_coords) > 1:
                                    final_segments.append([overlap_coords[0][1]+1,overlap_coords[1][0]-1])
                                else:
                                    if overlap_coords[0][1] == query_coords[query_index][1]:
                                        pass
                                    else:
                                        final_segments.append([overlap_coords[0][1]+1,query_coords[query_index][1]])
                            elif overlap_coords[0][0] > query_coords[query_index][0]:
                                sys.exit('should not be possible')
                        else:
                            #print(final_segments,overlap_coords)
                            if final_segments[-1][-1] < overlap_coords[i][0]-1:
                                if final_segments[-1][-1] <= overlap_coords[i-1][1]:
                                     final_segments.append([overlap_coords[i-1][1]+1,overlap_coords[i][0]-1])
                                else:
                                    final_segments.append([final_segments[-1][-1]+1,overlap_coords[i][0]-1])

                    if len(overlap_coords) != 0 and final_segments != [] and overlap_coords[-1][-1] < query_coords[query_index][1] and final_segments[-1][-1] < query_coords[query_index][1] :
                        final_segments.append([overlap_coords[i][1]+1,query_coords[query_index][1]])
                    dict_of_finals[query_index] = final_segments


                #investigate if there is enough sequence unmasked for a TD.
                total = 0
                for key in dict_of_finals.keys():
                    if dict_of_finals[key] != []:
                        for region in dict_of_finals[key]:
                            #print(region)
                            total = total + region[1]-region[0] +1
                #print(total)
                if total >= 25:
                    if split_line[9] not in dict_of_loci.keys():
                        dict_of_loci[split_line[9]] = []
                    dict_of_loci[split_line[9]].append([f"{split_line[13]}:{split_line[15]}-{split_line[16]}",total,split_line[-1],split_line[8]])
        remove_keys = []
        for key in dict_of_loci.keys():
            if len(dict_of_loci[key]) > 5:
                #print(dict_of_loci[key])
                remove_keys.append(key)

        for key in remove_keys:
            del dict_of_loci[key]
        if state == "query" or state == 'ref' and canine == 'Mischka':
            Elibible_loci.append(valid_locus)
            Final_TDs.append(len(dict_of_loci.keys()))
        
        print(canine,state)
        print("N loci with TD ", len(dict_of_loci.keys()))
        print("N valid loci = ",valid_locus)
        dict_of_dicts[f"{canine}_{state}"] = dict_of_loci

        list_of_output_lines = format_for_output(dict_of_loci,list_of_output_lines,dict_of_locus_name_info,canine,state)
with open("/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/Revised_All_Canines_List_2025_08_06/Transduction_Detection/Transduction_list.txt",'wt') as out_td:
    for item in list_of_output_lines:
        out_td.write(item)
print(Elibible_loci, Final_TDs)
print(sum(Elibible_loci)/len(Elibible_loci))
print(sum(Final_TDs)/len(Final_TDs))


Mischka query
N loci with TD  27
N valid loci =  55
Mischka ref
N loci with TD  32
N valid loci =  59
Nala query
N loci with TD  36
N valid loci =  56
Nala ref
N loci with TD  32
N valid loci =  60
Tasha query
N loci with TD  40
N valid loci =  72
Tasha ref
N loci with TD  32
N valid loci =  59
Zoey query
N loci with TD  40
N valid loci =  74
Zoey ref
N loci with TD  38
N valid loci =  73
Sandy query
N loci with TD  54
N valid loci =  81
Sandy ref
N loci with TD  37
N valid loci =  71
Clu query
N loci with TD  20
N valid loci =  44
Clu ref
N loci with TD  33
N valid loci =  65
[55, 59, 56, 72, 74, 81, 44] [27, 32, 36, 40, 40, 54, 20]
63.0
35.57142857142857


In [3]:
def analyze_dict_of_dicts(dictionary):
    #print(dictionary)
    for key in dict_of_dicts:
        max_TD_map = 0
        sample = dict_of_dicts[key] 
        for locus in sample.keys():
            if len(sample[locus]) > max_TD_map:
                max_TD_map = len(sample[locus])
            if len(sample[locus]) >= 5:
                print(locus)
                print(sample[locus])
        print("genome",key,".  Maximum number of G_WOLF locations that a TD maps to is",max_TD_map, "loci in the G_WOLF genome.")
def process_bedline(line):
    line_dict = {}
    line_dict["TD_start"] = int(line[1])
    line_dict["TD_end"] = int(line[2])
    line_dict['sample_name'] = line[3]
    line_dict['TD_blat_orientation'] = line[4]
    line_dict['RefLINE_start'] = int(line[6])
    line_dict['RefLINE_end'] = int(line[7])
    line_dict['RefLINE_orientation'] = line[13]
    line_dict['LocusName'] = f"{line[0]}:{line[1]}-{line[2]}"
    
    return line_dict
                
def process_bedtools_window_output(input_bed):
    with open(input_bed,'rt') as infile:
        final_dict = {}
        for line in infile:
            split_line = line.split()
            line_dict = process_bedline(split_line)
            if line_dict['sample_name'] not in final_dict.keys():
                final_dict[line_dict['sample_name']] = {}
            #check if LINE is entirely in the TD and skip if so
            if line_dict['RefLINE_start'] >= line_dict['TD_start'] and line_dict['RefLINE_end'] <= line_dict['TD_end']:
                final_dict[line_dict['sample_name']][line_dict['LocusName']] = ["Not_Adjacent",'N/A']
            elif  line_dict['RefLINE_start'] < line_dict['TD_start'] and line_dict['RefLINE_end'] > line_dict['TD_end']:
                final_dict[line_dict['sample_name']][line_dict['LocusName']] = ["Spanning_LINE1",'N/A']
                #should be disqualified
            elif line_dict['RefLINE_orientation'] == "+" and  line_dict['RefLINE_start'] < line_dict['TD_start']:
                final_dict[line_dict['sample_name']][line_dict['LocusName']] = ["Adjacent",'+']
                #TD is adjacent to LINE1
            elif line_dict['RefLINE_orientation'] == "C" and line_dict['RefLINE_end'] > line_dict['TD_end']:
                final_dict[line_dict['sample_name']][line_dict['LocusName']] = ["Adjacent","-"]
                #TD is adjacent to LINE1
            else:
                final_dict[line_dict['sample_name']][line_dict['LocusName']] = ["Not_Adjacent",'N/A']

       # print(final_dict)
        return final_dict
def check_for_parentless(dict_of_dicts):
    bed_loci = []
    for key in dict_of_dicts:
        sample = dict_of_dicts[key] 
        for loci in sample.keys():
            for locus in sample[loci]:
                bed_locus = (locus[0].replace(':','-').split('-'))
                bed_locus.append(key)
                bed_locus.append(locus[3])
                bed_locus = "\t".join(bed_locus) + "\n"
                bed_loci.append(bed_locus)
    bed_file = f"Revised_All_Canines_List_2025_08_06/Transduction_Detection/bed_loci.tmp"
    LINE1_in_mCan = "/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/RM_info_created_2025_08_06/mCanlor_LINE_RM.out.bed" 
    with open(bed_file,'wt') as outfile:
        for locus in bed_loci:
            outfile.write(locus)
    subprocess.run(f"bedtools sort -i {bed_file} > {bed_file}.sort",shell=True)
    subprocess.run(f"bedtools window -w 50 -a {bed_file}.sort -b {LINE1_in_mCan} > {bed_file}.intersected.bed",shell=True)
    
    dict_of_adjacent = process_bedtools_window_output(f"{bed_file}.intersected.bed")
    #print(dict_of_adjacent)
    

    multi = {}
    for key in dict_of_dicts.keys():
        parentless = 0
        parentful = 0
        for subkey in dict_of_dicts[key]:
            found_an_adjacent = False
            for locus_found in dict_of_dicts[key][subkey]:
                if locus_found[0] in dict_of_adjacent[key].keys():
                    if dict_of_adjacent[key][locus_found[0]][0] == "Adjacent" and dict_of_adjacent[key][locus_found[0]][1] == locus_found[3]:
                        found_an_adjacent = True #This requires 2 parameters. 1. There is a LINE-1 "adjacent" to the transduced sequence. 2. The sequence is in the expected orientation. This means that the orientation of the blat aligned transduced sequence matches the LINE-1 orientation. The reason this works is all extracted transduction sequences are "forward" in that they are downstream of the dimorphic LINE-1 from which they are identified. Thus, if the "parent" LINE-1 is forward, the blat alignmend would be expected to be forward as well.
                    #else:
                        #print(locus_found,subkey,dict_of_adjacent[key][locus_found[0]])
            if len(dict_of_dicts[key][subkey]) > 1:
                if "query" in key or ("ref" in key and subkey not in multi.keys()):
                    multi[subkey] = [key,subkey,len(dict_of_dicts[key][subkey]),found_an_adjacent]
            if found_an_adjacent == True:
                parentful +=1
            else:
                parentless +=1
        print("genome", key, ". Num loci parentless", parentless,". Num loci with parent",parentful)
    #return
    #print(multi)
    multi_parent,multi_parentless=0,0
    num_2s = 0
    num_3s = 0
    for key in multi.keys():
        if multi[key][-1] == True:
            multi_parent +=1
        else:
            multi_parentless +=1
        if multi[key][-2] == 2:
            num_2s +=1
        elif multi[key][-2] == 3:
            num_3s +=1
    print(f"The number of loci which corrspond to two or more G_WOLF loci and are parentless is {multi_parentless}. The number with parent is {multi_parent}.")
    print(f"Number of loci which correspon to 2 loci in G_WOLF {num_2s}. And 3 loci in G_WOLF {num_3s}.")
analyze_dict_of_dicts(dict_of_dicts)
check_for_parentless(dict_of_dicts)


genome Mischka_query .  Maximum number of G_WOLF locations that a TD maps to is 1 loci in the G_WOLF genome.
genome Mischka_ref .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Nala_query .  Maximum number of G_WOLF locations that a TD maps to is 1 loci in the G_WOLF genome.
genome Nala_ref .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Tasha_query .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Tasha_ref .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Zoey_query .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Zoey_ref .  Maximum number of G_WOLF locations that a TD maps to is 2 loci in the G_WOLF genome.
genome Sandy_query .  Maximum number of G_WOLF locations that a TD maps to is 3 loci in the G_WOLF genome.
genome Sandy_ref .  Maximum number of G_WOLF 

In [4]:
#generate bedfile of all locations that a transduction maps to in G_WOLF, regardless of which genome possesses the LINE-1 that the transduced seqeunce was originally identified in.
bedfile = "/nfs/turbo/umms-jmkidd/matt-projects/Former_KiddLabScratch_Data/inter-genome_comparisons/Aligning_Using_2.26/Revised_All_Canines_List_2025_08_06/Aggregated_TDs.bed"
outfile = open(bedfile,'wt')
header = '#Source_Chrom\tSource_Start\tSource_End\tL1_chrom\tL1_start\tL1_end\tFilled_Genome\tEmpty_Genome\tOther_Potential_Sources_From_Single_Blat_Alignment\n'
outfile.write(header)

def split_coords_into_bed(input_coords):
    #print(input_coords)
    input_coords = input_coords.replace(":","-").split("-")
    input_coords[1] = str(int(input_coords[1])-1)
    return input_coords

for subdict in dict_of_dicts.keys():
    print(subdict)
    split_subdict = subdict.split("_")
    
    #set names for filled and empty genomes respectively
    if split_subdict[1] == "query":
        filled_name = split_subdict[0]
        empty_name = "mCanlor"
    else:
        filled_name = "mCanlor"
        empty_name = split_subdict[0]
        
    for key in dict_of_dicts[subdict].keys():

        #split the info from the locus name
        split_name_info = dict_of_locus_name_info[subdict][key].split(",")
        if split_subdict[1] == "query":
            info = split_name_info[3:6]
        else:
            info = split_name_info[0:3]
        #here's the order chr_blat_hit, start_blat_hit, end_blat_hit, chr_L1_mcl, start_L1_mcl, end_L1_mcl, filled_name, empty_name, list_of_alternate_blats_which_meet_critera
        out = []
        if len(dict_of_dicts[subdict][key]) == 1:
            replaced_coords = split_coords_into_bed(dict_of_dicts[subdict][key][0][0])
            for item in replaced_coords:
                out.append(item)
            for item in info:
                out.append(item)
            out.append(filled_name)
            out.append(empty_name)
            out.append("N/A")
            out = "\t".join(out) + "\n"
            #if filled_name == "mCanlor" and empty_name != "Mischka":
            #        continue
            print(out)
            outfile.write(out)
        else:
            for i in range(len(dict_of_dicts[subdict][key])):
                out = []
                replaced_coords = split_coords_into_bed(dict_of_dicts[subdict][key][i][0])
                for item in replaced_coords:
                    out.append(item)
                for item in info:
                    out.append(item)
                out.append(filled_name)
                out.append(empty_name)
                other_loci = []
                for j in range(len(dict_of_dicts[subdict][key])):
                    if j != i:
                        other_loci.append(f"{dict_of_dicts[subdict][key][j][0]}-{subdict}")
                other_loci = ",".join(other_loci)
                out.append(other_loci)
                #if filled_name == "mCanlor" and empty_name != "Mischka":
                #    continue
                out = "\t".join(out) + "\n"
                print(out)
                outfile.write(out)
outfile.close()

Mischka_query
chr6	3644644	3644784	chr1	92781173	92781174	Mischka	mCanlor	N/A

chr3	16203024	16203124	chr12	48517936	48517936	Mischka	mCanlor	N/A

chr29	42742033	42742377	chr12	59643279	59643279	Mischka	mCanlor	N/A

chr8	62199237	62199570	chr13	41528448	41528448	Mischka	mCanlor	N/A

chr1	113873464	113873737	chr14	33262735	33262735	Mischka	mCanlor	N/A

chr29	35827310	35827463	chr14	21733517	21733517	Mischka	mCanlor	N/A

chr14	41470982	41471204	chr16	18265845	18265845	Mischka	mCanlor	N/A

chr1	44083499	44083696	chr17	43639362	43639362	Mischka	mCanlor	N/A

chr3	11176150	11176328	chr19	27324138	27324138	Mischka	mCanlor	N/A

chr35	19472332	19472369	chr22	20079366	20079366	Mischka	mCanlor	N/A

chr20	43395231	43395572	chr22	31986657	31986657	Mischka	mCanlor	N/A

chr7	19149270	19149321	chr23	7716235	7716235	Mischka	mCanlor	N/A

chrX	10914956	10915004	chr24	11270608	11270608	Mischka	mCanlor	N/A

chr36	24184035	24184111	chr25	35411128	35411128	Mischka	mCanlor	N/A

chr3	7869572	7869669	chr27	1782

In [5]:
#sort and merge the bedfile to identify which coordinates correspond to transductions in multiple independent LINE-1s
cmd = f"bedtools sort -i {bedfile} -header> {bedfile}.sort && bedtools merge -d 100 -c 4,5,6,7,8,9 -o collapse,collapse,collapse,collapse,collapse,collapse -i {bedfile}.sort > {bedfile}.sort.merge"
subprocess.run(cmd,shell=True)

loci_count = 0
with open(f"{bedfile}.sort.merge",'rt') as infile:
    for line in infile:
        #print(line)
        #if len(line.split()[-1].split(",")) > 10:
            #print(line)
        #    continue
        #else:
        #    continue
        #print(line)
        chroms = line.split()[3].split(',')
        starts = [int(x) for x in line.split()[4].split(',')]
        ends = [int(x) for x in line.split()[5].split(',')]
        
        if ((max(starts)-min(starts)) > 100 and (max(ends)-min(ends)) > 100) or len(set(chroms)) > 1:
            print(line)
        loci_count +=1
print(loci_count)
print(dict_of_dicts['Sandy_query'].keys())

#Final list of sequences which appear as transductions at the 3' of LINE-1s in multiple genomic locations.

chr1	94389848	94390362	chr36,chr21,chr19	16561524,45086689,48011185	16561524,45086689,48011185	Clu,Sandy,Sandy	mCanlor,mCanlor,mCanlor	chr7:21713473-21713969-Clu_query,chr7:21713470-21713891-Sandy_query,chr7:21713470-21713891-Sandy_query,chr21:45086235-45086689-Sandy_query

chr10	44788575	44788617	chr32,chr32,chr32,chr32,chr23,chr32,chr32	41540920,41540920,41540920,41540966,35778277,41540920,41540920	41541943,41541943,41541943,41541962,35778277,41541943,41541943	mCanlor,mCanlor,mCanlor,mCanlor,Sandy,mCanlor,mCanlor	Mischka,Nala,Tasha,Zoey,mCanlor,Sandy,Clu	N/A,N/A,N/A,N/A,chr32:41541869-41541922-Sandy_query,N/A,N/A

chr21	46673640	46673765	chr12,chr28	18535405,6240173	18535405,6240173	Sandy,Sandy	mCanlor,mCanlor	N/A,N/A

chr21	48348913	48349227	chr16,chr16,chr27	56700783,56700783,45999981	56707272,56707272,45999981	mCanlor,mCanlor,Sandy	Zoey,Sandy,mCanlor	N/A,N/A,chr16:56701039-56701098-Sandy_query

chr23	12061394	12061428	chr25,chrX	38579400,95835854	38579400,95835854	Tasha,Tasha	mCan